# Hardpoint Breakaway Tests - Recent History

This notebook evaluates the hardpoint breakaway tests performed on a given `day_obs`.  

In [ ]:
# Times Square Parameters
day_obs = 20250804  # YYYYMMDD

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import base64
import ipywidgets as W
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gs

from typing import Optional
from io import BytesIO
from IPython.display import display, clear_output

from lsst.ts.xml.enums.MTM1M3 import HardpointTest

from lsst.summit.utils.efdUtils import (
    getEfdData,
    getDayObsEndTime,
    getDayObsStartTime,
    makeEfdClient,
)

import hardpoint_recent_history as hpt

In [ ]:
# Number of hardpoints
n_hardpoints = 6

# Create an EFD client instance
client = makeEfdClient()

# Get the start and end times
start_time = getDayObsStartTime(day_obs)
end_time = getDayObsEndTime(day_obs)

In [ ]:
# Query the CSC level command that we can use to define the start of a
# breakaway test for a given strut.
test_command_df = getEfdData(
    client=client,
    topic="lsst.sal.MTM1M3.command_testHardpoint",
    columns=["hardpointActuator"],
    begin=start_time,
    end=end_time,
)

In [ ]:
# Query the status of the breakaway tests as they evolve.
test_status_df = getEfdData(
    client=client,
    topic="lsst.sal.MTM1M3.logevent_hardpointTestStatus",
    columns=[f"testState{i}" for i in range(n_hardpoints)],
    begin=start_time,
    end=end_time,
)

In [ ]:
def make_hardpoint_group_selector(
    commands: pd.DataFrame,
    statuses: pd.DataFrame,
    gap: str = "3min",
):
    """Select a group, simulate query, and plot status timeline boxes."""
    statuses = hpt.ensure_utc(statuses)
    commands = hpt.group_by_gaps(gap, hpt.ensure_utc(commands))
    summary = hpt.build_labels(commands, statuses)

    def _label(row: pd.Series) -> str:
        s = row["reference_time"].strftime("%Y-%m-%d %H:%M UTC")
        return f"Test run {int(row['group_id'])} | Started near {s} | {row['n']} cmds"

    options = [(_label(r), int(r["group_id"])) for _, r in summary.iterrows()]
    dd = W.Dropdown(description="Select test run:", options=options,
                    value=options[0][1] if options else None,
                    layout=W.Layout(width="650px"))

    out = W.Output()

    def show_group(gid: int):
        with out:
            
            # Clear the output and print a nice message for impacient people
            clear_output()
            print("Querying and processing data...")
            
            row = summary.loc[summary["group_id"] == gid].iloc[0]
            t0, t1 = row["start_time"], row["end_time"]
            
            sub_status = test_status_df.loc[t0:t1]
            fig, ax = plt.subplots(1, 1, figsize=(12, 4), constrained_layout=True)
            hpt.plot_status_timeline(ax, sub_status, t0, t1)
                        
            # display the figure using html
            display(W.HTML(f"<b>Group window:</b> {t0} → {t1} (UTC)"))
            display(fig)
            plt.close(fig)  # Ensures the plots are shown only once

            telemetry = hpt.query_hardpoints_telemetry(client, row)

            for hp in row["uniq_hp"]:
                fig, (ax_t, ax_s) = plt.subplots(1, 2, figsize=(12, 4), constrained_layout=True)
                hpt.plot_forces_timeline(ax_t, telemetry, hp-1)
                hpt.plot_stiffness(ax_s, telemetry, hp-1)
                display(W.HTML(f"<b>Hardpoint {hp}:</b>"))
                display(fig)
                plt.close(fig)
            

    def _on_change(change):
        if change["name"] == "value" and change["new"] is not None:
            show_group(change["new"])

    dd.observe(_on_change)
    display(dd, out)
    if dd.value is not None:
        show_group(dd.value)

    return dd, out


# Example usage ---
dd, out = make_hardpoint_group_selector(test_command_df, test_status_df)
